# 삼성카드 데이터 로드
- (삼성카드)가맹점 기반 지역별 소비행태의 A,C,E,F 칼럼 (기준년월 , 법정동명 , 업종명_중분류 , CNT) 
    - CNT = 전체매출건수
    
- 기준년월 : 'Date' , 법정동명 : 'Si', 업종명_중분류 : 'Shop', CNT : 'cnt'

In [1]:
import pandas as pd
import numpy as np

# 불러올 csv excel json 파일을 현재 주피터 파일 위치에 넣어둘 경우 , 파일명만 입력해도 불러올 수 있다.

data = pd.read_excel('(삼성카드)가맹점 기반 지역별 소비행태.xlsx' 
             , usecols = "A,C,E,F" , names = ['Date' , 'Si' , 'Shop' , 'cnt'])

In [2]:
# data display
data

,Date,Si,Shop,cnt
0,202003,전남 목포시 중앙동2가,음식점,79
1,202001,전북 군산시 경장동,취미,70
2,202003,경남 통영시 영운리,음식점,83
3,202003,전북 군산시 조촌동,아울렛,3643
4,202101,전북 군산시 조촌동,서적,17
...,...,...,...,...
7685,202002,전남 목포시 산정동,커피/음료,1444
7686,202103,전북 군산시 도암리,편의점,110
7687,202002,경남 통영시 정량동,차량관리,37
7688,202001,경남 통영시 화삼리,음식점,60


# slicing 함수
    삼성카드 데이터를 목포시, 통영시, 군산시로 분류해주는 함수
- area : str 
    - 목포시 , 통영시 , 군산시 기입
- name : str
    - mokpo , tongyeong , gunsan

    1. 입력된 area로 삼성카드 데이터를 시별로 분류 
    2. Si 칼럼의 문자열을 split으로 분할하여 시와 동으로 나눈다음 시는 Si 컬럼에 기입, 동은 Dong 칼럼을 추가
    3. 함수의 인자로 기입된 지역이름별 DataFrame 생성 (전역동적변수로 생성하여 외부에서도 사용이 가능)
    


### 사용된 내장함수

- globals()[''] : 전역-동적 변수이며, slicing 함수 외부에서도 이 변수를 사용하고
변수 이름을 함수 내부에서 지정하기 위해 사용.
- .query() : dataframe을 슬라이싱 하는데에있어 코드 간결하를 위해 사용
- .str.split() : dataframe의 object value 분석시 split으로 띄어쓰기마다 문자열을 분할하기 위해 사용


In [3]:
def slicing (area : str , name : str) :
    
    globals()[f'{name}_data'] = data.copy()
    print(f'{name}_data 가 생성되었습니다.')
    print()
    globals()[f'{name}_data'].insert(2 , 'Dong' , globals()[f'{name}_data']['Si'].str.split(' ' , expand = True)[2])
    globals()[f'{name}_indx'] = globals()[f'{name}_data'].query('Si.str.contains(@area)').index.to_list()
    globals()[f'{name}_data']['Si'] = area
    
    globals()[f'{name}_data'] = globals()[f'{name}_data'].loc[globals()[f'{name}_indx']].reset_index(drop = True)
    
    

In [4]:
slicing('목포시' , 'mokpo')
slicing('통영시' , 'tongyeong')
slicing('군산시' , 'gunsan')

mokpo_data 가 생성되었습니다.

tongyeong_data 가 생성되었습니다.

gunsan_data 가 생성되었습니다.



In [5]:
display(mokpo_data)
display(tongyeong_data)
display(gunsan_data)

,Date,Si,Dong,Shop,cnt
0,202003,목포시,중앙동2가,음식점,79
1,202002,목포시,상동,음식점,15003
2,202002,목포시,산정동,문구사무용품,107
3,202001,목포시,상동,보험,19
4,202102,목포시,대안동,병원,106
...,...,...,...,...,...
2086,202103,목포시,석현동,학원,28
2087,202001,목포시,상락동2가,건강보조,128
2088,202101,목포시,용해동,인테리어,89
2089,202002,목포시,대양동,인테리어,48


,Date,Si,Dong,Shop,cnt
0,202003,통영시,영운리,음식점,83
1,202001,통영시,삼화리,마트/할인점,118
2,202003,통영시,덕포리,골프장,49
3,202002,통영시,죽림리,편의점,4616
4,202002,통영시,정량동,인테리어,193
...,...,...,...,...,...
2043,202101,통영시,안정리,학원,13
2044,202103,통영시,도남동,건강보조,130
2045,202002,통영시,정량동,차량관리,37
2046,202001,통영시,화삼리,음식점,60


,Date,Si,Dong,Shop,cnt
0,202001,군산시,경장동,취미,70
1,202003,군산시,조촌동,아울렛,3643
2,202101,군산시,조촌동,서적,17
3,202101,군산시,문화동,마트/할인점,389
4,202101,군산시,경암동,차량관리,42
...,...,...,...,...,...
3546,202003,군산시,나운동,관람,49
3547,202101,군산시,미룡동,문구사무용품,45
3548,202102,군산시,문화동,의류,11
3549,202101,군산시,지경리,음식점,157


# top_and_bottom 함수
    slicing 함수에서 선언한 mokpo_data , tonyeong_data , gunsan_data의 매출건수가 가장높은 
    'Dong' 5곳과 가장낮은 'Dong' 5곳으로 매출건수(CNT)를 갖는 DataFrame을 생성하고 변수로 반환시켜주는 함수.
- df   : DataFrame  
    - mokpo_data , tonyeong_data , gunsan_data 기입
- name : str        
    - mokpo , tongyeong , gunsan

    1. Date 컬럼을 문자열로 형변환한 뒤, 2020과 2021을 분리
    2. 2020,2021의 'Dong'별 'cnt'(매출건수) 합계를 구하고 상위 5, 하위 5 데이터를 전역동적변수에 저장

In [6]:
# globals()[f'test_{name}_data']
# globals()[f'{name}_2020_spot']
def top_and_bottom (df , name : str) :
    df['Date'] = df['Date'].astype(str)
    data = df.copy()
        
    spot_2020 = data[data['Date'].str[:4] == '2020'].reset_index(drop = True)
    spot_2021 = data[data['Date'].str[:4] == '2021'].reset_index(drop = True)
    
    print('다음 데이터가 생성됩니다,')
    print(f'{name}_2020_hotspot')
    print(f'{name}_2021_hotspot')
    print(f'{name}_2020_coldspot')
    print(f'{name}_2021_coldspot')
    print()

    globals()[f'{name}_2020_hotspot'] = spot_2020.groupby(['Dong' ]).agg(sum).sort_values('cnt' , ascending = False).head(5)
    globals()[f'{name}_2021_hotspot'] = spot_2021.groupby(['Dong' ]).agg(sum).sort_values('cnt' , ascending = False).head(5)
    
    
    globals()[f'{name}_2020_coldspot'] = spot_2020.groupby(['Dong' ]).agg(sum).sort_values('cnt' , ascending = True).head(5)
    globals()[f'{name}_2021_coldspot'] = spot_2021.groupby(['Dong' ]).agg(sum).sort_values('cnt' , ascending = True).head(5)

In [7]:
top_and_bottom(mokpo_data     , 'mokpo')
top_and_bottom(tongyeong_data , 'tongyeong')
top_and_bottom(gunsan_data    , 'gunsan')

다음 데이터가 생성됩니다,
mokpo_2020_hotspot
mokpo_2021_hotspot
mokpo_2020_coldspot
mokpo_2021_coldspot

다음 데이터가 생성됩니다,
tongyeong_2020_hotspot
tongyeong_2021_hotspot
tongyeong_2020_coldspot
tongyeong_2021_coldspot

다음 데이터가 생성됩니다,
gunsan_2020_hotspot
gunsan_2021_hotspot
gunsan_2020_coldspot
gunsan_2021_coldspot



In [8]:
# 목포
display(mokpo_2020_hotspot)
display(mokpo_2021_hotspot)
display(mokpo_2020_coldspot)
display(mokpo_2021_coldspot)

,cnt
Dong,
상동,187219
옥암동,105203
산정동,95008
용당동,38709
석현동,38554


,cnt
Dong,
상동,168871
옥암동,102257
산정동,88462
석현동,41747
용당동,38771


,cnt
Dong,
유달동,10
축복동3가,24
보광동1가,37
광동2가,48
해안동2가,59


,cnt
Dong,
광동2가,11
금동1가,13
유달동,21
해안동3가,32
수강동2가,41


In [9]:
# 통영
display(tongyeong_2020_hotspot)
display(tongyeong_2021_hotspot)
display(tongyeong_2020_coldspot)
display(tongyeong_2021_coldspot)

,cnt
Dong,
죽림리,101084
무전동,70874
북신동,28422
미수동,13965
항남동,13169


,cnt
Dong,
죽림리,101172
무전동,68421
북신동,27100
정량동,13805
미수동,13495


,cnt
Dong,
저림리,10
우동리,36
매죽리,102
오륜리,107
덕포리,153


,cnt
Dong,
염호리,22
저림리,30
원산리,57
우동리,77
매죽리,119


In [10]:
# 군산
display(gunsan_2020_hotspot)
display(gunsan_2021_hotspot)
display(gunsan_2020_coldspot)
display(gunsan_2021_coldspot)

,cnt
Dong,
수송동,113525
나운동,107530
조촌동,62539
경암동,38001
소룡동,35600


,cnt
Dong,
수송동,104330
나운동,98980
조촌동,67223
경암동,39537
소룡동,33471


,cnt
Dong,
어청도리,16
야미도리,22
옥정리,25
술산리,27
운회리,33


,cnt
Dong,
선양동,10
학당리,11
원우리,12
운회리,15
야미도리,21


# 주소 찾기
### 추가 데이터 load
- 행정_법정동 중심좌표 파일의 A,F,G 코드 (코드 , 위도 , 경도)
- (삼성카드)가맹점 기반 지역별 소비행태의 B,C 칼럼 (법정동코드 , 법정동명) 
- 코드 : 'Code' , 위도 : 'lat' , 경도 : 'lng'
- 기준년월 : 'Date' , 법정동명 : 'Si', 업종명_중분류 : 'Shop', CNT : 'cnt'

In [11]:
address = pd.read_excel('행정_법정동 중심좌표.xlsx' 
                       , usecols = "A , F , G" , names = ['Code' , 'lat' , 'lng'])
address_code = pd.read_excel('(삼성카드)가맹점 기반 지역별 소비행태.xlsx' 
             , usecols = "B , C" , names = ['Code' , 'Dong'])

display(address_code)
display(address)
address_code['Dong'] = address_code['Dong'].str.split(' ' , expand=True)[2]

,Code,Dong
0,4611013700,전남 목포시 중앙동2가
1,4513013500,전북 군산시 경장동
2,4822025021,경남 통영시 영운리
3,4513013400,전북 군산시 조촌동
4,4513013400,전북 군산시 조촌동
...,...,...
7685,4611010200,전남 목포시 산정동
7686,4513037024,전북 군산시 도암리
7687,4822010900,경남 통영시 정량동
7688,4822031021,경남 통영시 화삼리


,Code,lat,lng
0,1111051500,37.584009,126.970626
1,1111053000,37.575408,126.965944
2,1111054000,37.590758,126.980996
3,1111055000,37.594768,126.965574
4,1111056000,37.613029,126.974485
...,...,...,...
22214,5013032022,33.348068,126.832421
22215,5013032023,33.393144,126.794822
22216,5013032024,33.354784,126.771072
22217,5013032025,33.321147,126.798867


# geodata_lat_lng 함수
    top_and_bottom에서 선언된 dataframe에 법정동코드 , 위도 , 경도 값을 결합하여 Dataframe을 
    변수에 넣어 반환하는 함수
- spot : DataFrame  
    - mokpo_2020_hotspot, mokpo_2021_hotspot, mokpo_2020_coldspot, mokpo_2021_coldspot ...
- name : str        
    - mokpo_2020_hotspot, mokpo_2021_hotspot, mokpo_2020_coldspot, mokpo_2021_coldspot ...

    1. spot으로 불러온 DataFrame에 address_code를 'Dong' 칼럼 기준으로 결합하고 'Dong' 칼럼 기준으로 중복제거
    2. 1의 dataframe에 address를 'Code' 칼럼 기준으로 결합한 dataframe을 전역동적변수로 선언하여 변수 반환

In [12]:
def geodata_lat_lng (spot , name : str) :
    data = pd.merge(spot , address_code , how = 'left' , on = 'Dong').drop_duplicates(['Dong'])
    globals()[f'{name}_frm'] = pd.merge(data , address , how = 'inner' , on = 'Code')

# mokpo_2020_hotspot_code = pd.merge(mokpo_2020_hotspot , address_code , how = 'left' , on = 'Dong').drop_duplicates(['Dong'])
# mokpo_2020_hotspot_code_frm = pd.merge(mokpo_2020_hotspot_code , address , how = 'inner' , on = 'Code')

In [13]:
# 목포

geodata_lat_lng(mokpo_2020_hotspot , 'mokpo_2020_hotspot')
display(mokpo_2020_hotspot_frm)
geodata_lat_lng(mokpo_2021_hotspot , 'mokpo_2021_hotspot')
display(mokpo_2021_hotspot_frm)

geodata_lat_lng(mokpo_2020_coldspot , 'mokpo_2020_coldspot')
display(mokpo_2020_coldspot_frm)
geodata_lat_lng(mokpo_2021_coldspot , 'mokpo_2021_coldspot')
display(mokpo_2021_coldspot_frm)



,Dong,cnt,Code,lat,lng
0,상동,187219,4611015800,34.804528,126.421473
1,옥암동,105203,4611016400,34.808392,126.439544
2,산정동,95008,4611010200,34.799329,126.387817
3,용당동,38709,4611010100,34.804670,126.396526
4,석현동,38554,4611016000,34.823682,126.423886


,Dong,cnt,Code,lat,lng
0,상동,168871,4611015800,34.804528,126.421473
1,옥암동,102257,4611016400,34.808392,126.439544
2,산정동,88462,4611010200,34.799329,126.387817
3,석현동,41747,4611016000,34.823682,126.423886
4,용당동,38771,4611010100,34.804670,126.396526


,Dong,cnt,Code,lat,lng
0,유달동,10,4611013200,34.786096,126.378970
1,축복동3가,24,4611012800,34.788706,126.387668
2,보광동1가,37,4611012900,34.787023,126.388493
3,광동2가,48,4611012000,34.788566,126.389122
4,해안동2가,59,4611014300,34.782503,126.381763


,Dong,cnt,Code,lat,lng
0,광동2가,11,4611012000,34.788566,126.389122
1,금동1가,13,4611015000,34.784425,126.383122
2,유달동,21,4611013200,34.786096,126.378970
3,해안동3가,32,4611014400,34.781929,126.381194
4,수강동2가,41,4611014100,34.786179,126.385025


In [14]:
# 통영

geodata_lat_lng(tongyeong_2020_hotspot , 'tongyeong_2020_hotspot')
display(tongyeong_2020_hotspot_frm)
geodata_lat_lng(tongyeong_2021_hotspot , 'tongyeong_2021_hotspot')
display(tongyeong_2021_hotspot_frm)

geodata_lat_lng(tongyeong_2020_coldspot , 'tongyeong_2020_coldspot')
display(tongyeong_2020_coldspot_frm)
geodata_lat_lng(tongyeong_2021_coldspot , 'tongyeong_2021_coldspot')
display(tongyeong_2021_coldspot_frm)



,Dong,cnt,Code,lat,lng
0,죽림리,101084,4822034024,34.880595,128.414139
1,무전동,70874,4822011100,34.857503,128.430068
2,북신동,28422,4822011000,34.853629,128.427785
3,미수동,13965,4822011500,34.827532,128.403210
4,항남동,13169,4822010400,34.842105,128.422649


,Dong,cnt,Code,lat,lng
0,죽림리,101172,4822034024,34.880595,128.414139
1,무전동,68421,4822011100,34.857503,128.430068
2,북신동,27100,4822011000,34.853629,128.427785
3,정량동,13805,4822010900,34.847147,128.432199
4,미수동,13495,4822011500,34.827532,128.403210


,Dong,cnt,Code,lat,lng
0,저림리,10,4822025030,34.748013,128.402325
1,우동리,36,4822034022,34.917381,128.407978
2,매죽리,102,4822036028,34.673488,128.556879
3,오륜리,107,4822033025,34.906092,128.330595
4,덕포리,153,4822034021,34.913039,128.422544


,Dong,cnt,Code,lat,lng
0,염호리,22,4822036022,34.804053,128.481651
1,저림리,30,4822025030,34.748013,128.402325
2,원산리,57,4822033021,34.935900,128.358304
3,우동리,77,4822034022,34.917381,128.407978
4,매죽리,119,4822036028,34.673488,128.556879


In [15]:
# 군산

geodata_lat_lng(gunsan_2020_hotspot , 'gunsan_2020_hotspot')
display(gunsan_2020_hotspot_frm)
geodata_lat_lng(gunsan_2021_hotspot , 'gunsan_2021_hotspot')
display(gunsan_2021_hotspot_frm)

geodata_lat_lng(gunsan_2020_coldspot , 'gunsan_2020_coldspot')
display(gunsan_2020_coldspot_frm)
geodata_lat_lng(gunsan_2021_coldspot , 'gunsan_2021_coldspot')
display(gunsan_2021_coldspot_frm)



,Dong,cnt,Code,lat,lng
0,수송동,113525,4513014100,35.966811,126.716726
1,나운동,107530,4513014400,35.966556,126.697073
2,조촌동,62539,4513013400,35.971973,126.738054
3,경암동,38001,4513013600,35.978672,126.729567
4,소룡동,35600,4513014600,35.975439,126.669758


,Dong,cnt,Code,lat,lng
0,수송동,104330,4513014100,35.966811,126.716726
1,나운동,98980,4513014400,35.966556,126.697073
2,조촌동,67223,4513013400,35.971973,126.738054
3,경암동,39537,4513013600,35.978672,126.729567
4,소룡동,33471,4513014600,35.975439,126.669758


,Dong,cnt,Code,lat,lng
0,어청도리,16,4513039023,36.118991,125.981152
1,야미도리,22,4513039027,35.841656,126.490999
2,옥정리,25,4513025021,35.935654,126.702008
3,술산리,27,4513033025,35.955791,126.864368
4,운회리,33,4513036022,35.970865,126.781932


,Dong,cnt,Code,lat,lng
0,선양동,10,4513011200,35.980830,126.712576
1,학당리,11,4513032026,35.917782,126.769537
2,원우리,12,4513032027,35.932614,126.777920
3,운회리,15,4513036022,35.970865,126.781932
4,야미도리,21,4513039027,35.841656,126.490999


# 지도 시각화
- 목포, 통영, 군산의 위도와 경도 정보로 각 folium 지도 생성
- 위도, 경도 정보가 포함된 데이터프레임으로 반복문을 사용하여 각 위치를 지도에 CircleMarker로 표시(핫스팟: red, 콜드스팟: blue)

In [16]:
import json as j
import folium
import pandas as pd

In [17]:
# tiles : 'Stamen Toner'" OpenStreetMap" 'cartodbpositron'

### 목포

In [18]:
map_01 = folium.Map(location=[34.8118351 , 126.3921664], zoom_start=13)

for i in mokpo_2020_hotspot_frm.index :
    lat = mokpo_2020_hotspot_frm.loc[i , 'lat']
    lng = mokpo_2020_hotspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'red'     ,
        color = 'red' ,
        fill_opacity = 0.7
    ).add_to(map_01)
    
for i in mokpo_2020_coldspot_frm.index :
    lat = mokpo_2020_coldspot_frm.loc[i , 'lat']
    lng = mokpo_2020_coldspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'blue'     ,
        color = 'blue' ,
        fill_opacity = 0.7
    ).add_to(map_01)
map_01



In [19]:
map_04 = folium.Map(location=[34.8118351 , 126.3921664], zoom_start=13)

for i in mokpo_2021_hotspot_frm.index :
    lat = mokpo_2021_hotspot_frm.loc[i , 'lat']
    lng = mokpo_2021_hotspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'red'     ,
        color = 'red' ,
        fill_opacity = 0.7
    ).add_to(map_04)
    
for i in mokpo_2021_coldspot_frm.index :
    lat = mokpo_2021_coldspot_frm.loc[i , 'lat']
    lng = mokpo_2021_coldspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'blue'     ,
        color = 'blue' ,
        fill_opacity = 0.7
    ).add_to(map_04)
map_04



### 통영


In [20]:
map_02 = folium.Map(location=[34.8544227 , 128.433182], zoom_start=13)

for i in tongyeong_2020_hotspot_frm.index :
    lat = tongyeong_2020_hotspot_frm.loc[i , 'lat']
    lng = tongyeong_2020_hotspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'red'     ,
        color = 'red' ,
        fill_opacity = 0.7
    ).add_to(map_02)
    
for i in tongyeong_2020_coldspot_frm.index :
    lat = tongyeong_2020_coldspot_frm.loc[i , 'lat']
    lng = tongyeong_2020_coldspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'blue'     ,
        color = 'blue' ,
        fill_opacity = 0.7
    ).add_to(map_02)
map_02



In [21]:
map_05 = folium.Map(location=[34.8544227 , 128.433182], zoom_start=13)

for i in tongyeong_2021_hotspot_frm.index :
    lat = tongyeong_2021_hotspot_frm.loc[i , 'lat']
    lng = tongyeong_2021_hotspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'red'     ,
        color = 'red' ,
        fill_opacity = 0.7
    ).add_to(map_05)
    
for i in tongyeong_2021_coldspot_frm.index :
    lat = tongyeong_2021_coldspot_frm.loc[i , 'lat']
    lng = tongyeong_2021_coldspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'blue'     ,
        color = 'blue' ,
        fill_opacity = 0.7
    ).add_to(map_05)
map_05



### 군산

In [22]:
map_03 = folium.Map(location=[35.9676772 , 126.7366293], zoom_start=11)

for i in gunsan_2020_hotspot_frm.index :
    lat = gunsan_2020_hotspot_frm.loc[i , 'lat']
    lng = gunsan_2020_hotspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'red'     ,
        color = 'red' ,
        fill_opacity = 0.7
    ).add_to(map_03)
    
for i in gunsan_2020_coldspot_frm.index :
    lat = gunsan_2020_coldspot_frm.loc[i , 'lat']
    lng = gunsan_2020_coldspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'blue'     ,
        color = 'blue' ,
        fill_opacity = 0.7
    ).add_to(map_03)
map_03



In [23]:
map_06 = folium.Map(location=[35.9676772 , 126.7366293], zoom_start=11)

for i in gunsan_2021_hotspot_frm.index :
    lat = gunsan_2021_hotspot_frm.loc[i , 'lat']
    lng = gunsan_2021_hotspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'red'     ,
        color = 'red' ,
        fill_opacity = 0.7
    ).add_to(map_06)
    
for i in gunsan_2021_coldspot_frm.index :
    lat = gunsan_2021_coldspot_frm.loc[i , 'lat']
    lng = gunsan_2021_coldspot_frm.loc[i , 'lng']
    folium.CircleMarker(
        location = [lat , lng] ,
        fill_color = 'blue'     ,
        color = 'blue' ,
        fill_opacity = 0.7
    ).add_to(map_06)
map_06

